# NEC winter-school: second exercise

In this exercise you will learn how to create and to train a CNN that can classify digits. 

During this exercise we will learn the basics of Tensorflow and we will get some intuition on how the CNN works.

Note: this notebook is based on the following tutorial https://www.tensorflow.org/tutorials/quickstart/advanced

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np                     # Basic array operation library
import matplotlib.pyplot as plt        # For plotting and displaying
%matplotlib inline

Load dataset:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0         # Rescale the image values to [0,1]

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Display a grid of 5x5 digits:

In [3]:
# Add code


Print the 25 first labels:

In [4]:
# Fill me with code


Create dataset feeder:

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Create the model.

Here you have to complete the code to create your model. 

*Requirement*, the dimensionality of the last layer has to be 2.

In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # Fill me with code


    def call(self, x):
        # Fill me with code

        return x

    def get_emb(self, x):
        '''This function should return a 2d latent representation'''
        # Fill me with code
        
        return x


Now we compile the model and train it. For simplicity, we will use the Keras interfaces, but if you want to have more control over the training, please follow the tutorial of the link provided at the beginning of this notebook.

In [7]:
# Create an instance of the model
model = MyModel()

model.compile(optimizer='adam',                         # Optimizer
              loss='sparse_categorical_crossentropy',   # Loss function
              metrics=['accuracy'])                     # Test metric to control that everything is working well

In [8]:
model.fit(x_train, y_train, epochs=5)

Let's save the model.

In [9]:
model.save_weights('mnist_model.h5')

Now we test our model in unseen data.

For this particular case, we use the "test" split.

Typically in machine learning, we work with 3 splits: train, validation, and test.
* The train split is the one used for fitting the model.
* The validation is used during the training to fit the hyperparameter of the model.
* The test split is the one used to report the performance of the model.

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

If you got a 0.96 of accuracy over the test set or so, it is a sign that your model is trained properly.

Now, plot in a grid some of the filters that your CNN learned.

In [11]:
# Add code


Create a 2d scatter plot with the 2d embedding of the test samples. Plot them together with the labels.

In [12]:
# Fill me with code
